<h1><center>Training our classifier model </center></h1>

Importing the required libraries and modules:

In [10]:
import os
import praw
import pandas as pd
import datetime as dt
import logging
import numpy as np
from numpy import random
import gensim
import nltk
nltk.download('all')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.cross_validation import train_test



[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /home/pranay/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /home/pranay/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /home/pranay/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     /home/pranay/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     /home/pranay/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     /home/pranay/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package

ModuleNotFoundError: No module named 'sklearn.cross_validation'

Loading the cleaned dataset and visualising the dimensions of it:

In [12]:
flares = ["AskIndia", "Non-Political", "[R]eddiquette", "Scheduled", "Photography", "Science/Technology", "Politics", "Business/Finance", "Policy/Economy", "Sports", "Food", "AMA"]


rindi = pd.read_csv('Data/cleaned.csv')
rindi.head()

FileNotFoundError: [Errno 2] File Data/cleaned.csv does not exist: 'Data/cleaned.csv'

In [108]:

def get_date(created):
    return dt.datetime.fromtimestamp(created)

def string_form(value):
    return str(value)

def clean_text(text):
     from sklearn.linear_model import SGDClassifier
     text = BeautifulSoup(text, "lxml").text
     text = text.lower()
     text = REPLACE_BY_SPACE_RE.sub(' ', text)
     text = BAD_SYMBOLS_RE.sub('', text)
     text = ' '.join(word for word in text.split() if word not in STOPWORDS)
     return text


In [109]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))



In [110]:
rindi['title'] = rindi['title'].apply(string_form)
rindi['body'] = rindi['body'].apply(string_form)
rindi['comments'] = rindi['comments'].apply(string_form)

rindi['title'] = rindi['title'].apply(clean_text)
rindi['body'] = rindi['body'].apply(clean_text)
rindi['comments'] = rindi['comments'].apply(clean_text)

feature_combine = rindi["title"] + rindi["comments"] + rindi["url"] + rindi['body']
rindi = rindi.assign(feature_combine = feature_combine)

/home/pranay/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:314: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


In [111]:
def nb_classifier(X_train, X_test, y_train, y_test):
  
  from sklearn.naive_bayes import MultinomialNB


  nb = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('clf', MultinomialNB()),
                ])
  nb.fit(X_train, y_train)

  y_pred = nb.predict(X_test)

  print('accuracy %s' % accuracy_score(y_pred, y_test))
  #print(classification_report(y_test, y_pred,target_names=flares))


In [112]:
def linear_svm(X_train, X_test, y_train, y_test):
  
  from sklearn.linear_model import SGDClassifier

  sgd = Pipeline([('vect', CountVectorizer()),
                  ('tfidf', TfidfTransformer()),
                  ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
                 ])
  sgd.fit(X_train, y_train)

  y_pred = sgd.predict(X_test)

  print('accuracy %s' % accuracy_score(y_pred, y_test))
  #print(classification_report(y_test, y_pred,target_names=flares))

In [113]:
    
def logisticreg(X_train, X_test, y_train, y_test):

    from sklearn.linear_model import LogisticRegression

    logreg = Pipeline([('vect', CountVectorizer()),
                  ('tfidf', TfidfTransformer()),
                  ('clf', LogisticRegression(n_jobs=1, C=1e30)),
                 ])
    logreg.fit(X_train, y_train)

    y_pred = logreg.predict(X_test)

    print('accuracy %s' % accuracy_score(y_pred, y_test))
   # print(classification_report(y_test, y_pred,target_names=flairs))


In [114]:

def randomforest(X_train, X_test, y_train, y_test):
  
  from sklearn.ensemble import RandomForestClassifier
  
  ranfor = Pipeline([('vect', CountVectorizer()),
                  ('tfidf', TfidfTransformer()),
                  ('clf', RandomForestClassifier(n_estimators = 1000, random_state = 42)),
                 ])
  ranfor.fit(X_train, y_train)

  y_pred = ranfor.predict(X_test)

  print('accuracy %s' % accuracy_score(y_pred, y_test))
  #print(classification_report(y_test, y_pred,target_names=flairs))


In [115]:

def mlpclassifier(X_train, X_test, y_train, y_test):
  
  from sklearn.neural_network import MLPClassifier
  
  mlp = Pipeline([('vect', CountVectorizer()),
                  ('tfidf', TfidfTransformer()),
                  ('clf', MLPClassifier(hidden_layer_sizes=(30,30,30))),
                 ])
  mlp.fit(X_train, y_train)

  y_pred = mlp.predict(X_test)

  print('accuracy %s' % accuracy_score(y_pred, y_test))
  #print(classification_report(y_test, y_pred,target_names=flairs))

In [1]:
def train_test(X,y):
 
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state = 42)

  print("Results of Naive Bayes Classifier")
  nb_classifier(X_train, X_test, y_train, y_test)
  print("Results of Linear Support Vector Machine")
  linear_svm(X_train, X_test, y_train, y_test)
  print("Results of Logistic Regression")
  logisticreg(X_train, X_test, y_train, y_test)
  print("Results of Random Forest")
  randomforest(X_train, X_test, y_train, y_test)
  print("Results of MLP Classifier")
  mlpclassifier(X_train, X_test, y_train, y_test)

In [117]:
cat = rindi.flair

V = rindi.feature_combine
W = rindi.comments
X = rindi.title
Y = rindi.body
Z = rindi.url

print("Flair Detection using Title as Feature:")
print("\n")
train_test(X,cat)
print("\n")
print("Flair Detection using Body as Feature:")
print("\n")
train_test(Y,cat)
print("\n")
print("Flair Detection using URL as Feature:")
print("\n")
train_test(Z,cat)
print("\n")
print("Flair Detection using Comments as Feature:")
print("\n")
train_test(W,cat)
print("\n")
print("Flair Detection using combined features as Feature:")
print("\n")
train_test(V,cat)


Flair Detection using Title as Feature:


accuracy 0.8113207547169812


Flair Detection using Body as Feature:


accuracy 0.8278301886792453


Flair Detection using URL as Feature:


accuracy 0.7523584905660378


Flair Detection using Comments as Feature:


accuracy 0.4056603773584906


Flair Detection using combined features as Feature:


accuracy 0.8325471698113207


In [92]:
import pickle

In [122]:
pickle.dump(linear_svm, open('model.pkl','wb'))